### Importing libraries

In [37]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from datetime import datetime
%matplotlib inline

### Loading the data

In [38]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
Submission=pd.read_csv('SampleSubmission.csv')
train.sample(5)

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
107822,train_id_107822,2018-08-02,UG 0009,DJE,TUN,2018-08-02 16:15:00,2018-08-02 17.15.00,SCH,UG AT7LBE,0.0
3016,train_id_3016,2016-06-18,TU 0541,DUS,NBE,2016-06-18 10:30:00,2016-06-18 13.15.00,ATA,TU 32AIMF,29.0
10400,train_id_10400,2016-07-07,UG 1311,CDG,TUN,2016-07-07 17:05:00,2016-07-07 19.20.00,DEP,TU CR9ISA,20.0
32049,train_id_32049,2016-02-05,UG 0002,TUN,DJE,2016-02-05 06:15:00,2016-02-05 07.15.00,SCH,UG AT7LBD,0.0
77918,train_id_77918,2018-07-11,TU 0613,DSS,TUN,2018-07-11 22:20:00,2018-07-12 03.00.00,ATA,TU 320IMR,40.0


In [39]:
test.sample(5)

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC
7904,test_id_7904,2018-09-20,TU 0744,TUN,FRA,2018-09-20 08:50:00,2018-09-20 11.25.00,ATA,TU 320IMU
7903,test_id_7903,2018-09-20,TU 0362,TUN,BLQ,2018-09-20 08:25:00,2018-09-20 10.00.00,ATA,TU 320IMV
3205,test_id_3205,2017-02-19,TU 0635,ORY,DJE,2017-02-19 17:20:00,2017-02-19 20.10.00,ATA,TU 736IOP
9126,test_id_9126,2018-09-18,UG 0008,TUN,DJE,2018-09-18 14:30:00,2018-09-18 15.30.00,SCH,UG AT7LBE
2490,test_id_2490,2016-05-12,WKL 0000,TUN,TUN,2016-05-12 22:00:00,2016-05-13 01.00.00,SCH,TU 320IMT


In [40]:
train.STATUS.unique()

array(['ATA', 'DEP', 'RTR', 'SCH', 'DEL'], dtype=object)

### Variable definitions
DATOP - Date of flight

FLTID - Flight number

DEPSTN - Departure point

ARRSTN - Arrival point

STD - Scheduled Time departure

STA - Scheduled Time arrival

STATUS - Flight status

ETD - Expected Time departure

ETA - Expected Time arrival

ATD - Actual Time of Departure

ATA - Actual Time of arrival

AC - Aircraft Code

In [41]:
print('dimension of the train set: ',train.shape)
print('dimension of the test set : ',test.shape)

dimension of the train set:  (107833, 10)
dimension of the test set :  (9333, 9)


### Data Preprocessing

In [42]:
#Combining train and test set for easy preprocessing:

ntrain = train.shape[0] # will be used to split train and test set from the combined dataframe
data = pd.concat((train, test)).reset_index(drop=True)
print(f'The shape of the combined dataframe is: {data.shape}')

The shape of the combined dataframe is: (117166, 10)


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117166 entries, 0 to 117165
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      117166 non-null  object 
 1   DATOP   117166 non-null  object 
 2   FLTID   117166 non-null  object 
 3   DEPSTN  117166 non-null  object 
 4   ARRSTN  117166 non-null  object 
 5   STD     117166 non-null  object 
 6   STA     117166 non-null  object 
 7   STATUS  117166 non-null  object 
 8   AC      117166 non-null  object 
 9   target  107833 non-null  float64
dtypes: float64(1), object(9)
memory usage: 8.9+ MB


In [44]:
#Convert to datetime
data['STA']=data['STA'].map(lambda x:x.replace('.',':'))
data[['DATOP','STD','STA']]=data[['DATOP','STD','STA']].apply(pd.to_datetime,errors='coerce')

In [45]:
#data['DATOP_day']=data['DATOP'].dt.day
#data['DATOP_month']=data['DATOP'].dt.month
#data['DATOP_year']=data['DATOP'].dt.year
data.drop(['DATOP'],axis=1,inplace=True)

In [46]:
def date_split(column):
    """
    split DateTime Data to create multiple feature
    """
    data[column+'_day']=data[column].dt.day
    data[column+'_month']=data[column].dt.month
    data[column+'_year']=data[column].dt.year
    data[column+'_hours']=data[column].dt.hour
    #data[column+'_minutes']=data[column].dt.minute
    #data[column+'_seconds']=data[column].dt.second

In [47]:
#apply date split to STD and STA
date_split('STD')
date_split('STA')
data.drop(['STD','STA'],axis=1,inplace=True)

##### Category columns

In [48]:
cat_cols=['FLTID','DEPSTN','ARRSTN','AC','STATUS']

In [49]:
# Change columns to their respective datatypes
data[cat_cols]=data[cat_cols].astype('category')

In [50]:
#Number of unique values per categorical columns
for col in cat_cols:
    print(col,data[col].nunique())

FLTID 1912
DEPSTN 134
ARRSTN 130
AC 70
STATUS 5


In [51]:
data=pd.get_dummies(data =data, columns = [cat_cols[-1]])

In [52]:
# Encode categorical features
#Label Encoding
from sklearn.preprocessing import LabelEncoder
for col in cat_cols[:-1]:
    data[col]=LabelEncoder().fit_transform(data[col])

In [53]:
data.head()

,ID,FLTID,DEPSTN,ARRSTN,AC,target,STD_day,STD_month,STD_year,STD_hours,STA_day,STA_month,STA_year,STA_hours,STATUS_ATA,STATUS_DEL,STATUS_DEP,STATUS_RTR,STATUS_SCH
0,train_id_0,239,32,121,47,260.0,3,1,2016,10,3,1,2016,12,1,0,0,0,0
1,train_id_1,266,88,121,29,20.0,13,1,2016,15,13,1,2016,16,1,0,0,0,0
2,train_id_2,93,125,59,47,0.0,16,1,2016,4,16,1,2016,6,1,0,0,0,0
3,train_id_3,173,38,93,51,0.0,17,1,2016,14,17,1,2016,17,1,0,0,0,0
4,train_id_4,145,125,4,35,22.0,17,1,2016,14,17,1,2016,15,1,0,0,0,0


In [54]:
data.shape

(117166, 19)

### Training and making predictions

In [55]:
# Separate train and test data from the combined dataframe
train_df =data[:ntrain]
test_df =data[ntrain:]

# Check the shapes of the split dataset
train_df.shape, test_df.shape

((107833, 19), (9333, 19))

In [56]:
main_cols=data.columns.difference(['ID','target'])
X=train_df[main_cols]
y=train_df.target

In [98]:
# Split data into train and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=44)

#### Linear Regression

In [99]:
from sklearn.linear_model import LinearRegression

model=LinearRegression()
model.fit(X_train,y_train)
print('done')

done


In [100]:
y_preds=model.predict(X_test)

In [102]:
model.score(X_test,y_test)

0.04089814787595847

In [103]:
rms = mean_squared_error(y_test, y_preds, squared=False)
rms

109.89146216334746

#### Catboost

In [104]:
from catboost import CatBoostRegressor

In [107]:
model = CatBoostRegressor(iterations=1500,learning_rate=0.11, depth=13,l2_leaf_reg=0.7, loss_function='RMSE')
model =model.fit(X_train,y_train)

0:	learn: 117.1690162	total: 162ms	remaining: 4m 3s
1:	learn: 116.3074059	total: 318ms	remaining: 3m 58s
2:	learn: 115.3754319	total: 477ms	remaining: 3m 58s
3:	learn: 114.6767604	total: 630ms	remaining: 3m 55s
4:	learn: 114.0976110	total: 800ms	remaining: 3m 59s
5:	learn: 113.6057398	total: 1.01s	remaining: 4m 12s
6:	learn: 113.1174295	total: 1.23s	remaining: 4m 23s
7:	learn: 112.5917765	total: 1.42s	remaining: 4m 24s
8:	learn: 112.1895463	total: 1.59s	remaining: 4m 23s
9:	learn: 111.8588261	total: 1.77s	remaining: 4m 23s
10:	learn: 111.3093164	total: 1.94s	remaining: 4m 22s
11:	learn: 111.0482594	total: 2.1s	remaining: 4m 20s
12:	learn: 110.7301906	total: 2.26s	remaining: 4m 18s
13:	learn: 110.3727673	total: 2.42s	remaining: 4m 16s
14:	learn: 110.0205881	total: 2.58s	remaining: 4m 15s
15:	learn: 109.7476806	total: 2.75s	remaining: 4m 14s
16:	learn: 109.4353129	total: 3.07s	remaining: 4m 27s
17:	learn: 109.1401052	total: 3.25s	remaining: 4m 28s
18:	learn: 108.6498334	total: 3.45s	rema

154:	learn: 85.7072755	total: 30.1s	remaining: 4m 20s
155:	learn: 85.6062350	total: 30.2s	remaining: 4m 20s
156:	learn: 85.5315481	total: 30.4s	remaining: 4m 20s
157:	learn: 85.4087775	total: 30.6s	remaining: 4m 19s
158:	learn: 85.3091049	total: 30.8s	remaining: 4m 19s
159:	learn: 85.2137787	total: 31s	remaining: 4m 19s
160:	learn: 85.1368524	total: 31.2s	remaining: 4m 19s
161:	learn: 85.0543379	total: 31.3s	remaining: 4m 18s
162:	learn: 84.9555781	total: 31.5s	remaining: 4m 18s
163:	learn: 84.8641050	total: 31.7s	remaining: 4m 18s
164:	learn: 84.7515932	total: 31.9s	remaining: 4m 17s
165:	learn: 84.5116055	total: 32s	remaining: 4m 17s
166:	learn: 84.3729094	total: 32.2s	remaining: 4m 16s
167:	learn: 84.3060821	total: 32.4s	remaining: 4m 16s
168:	learn: 84.1713976	total: 32.5s	remaining: 4m 16s
169:	learn: 84.0537087	total: 32.7s	remaining: 4m 15s
170:	learn: 83.9713747	total: 32.9s	remaining: 4m 15s
171:	learn: 83.8299038	total: 33s	remaining: 4m 15s
172:	learn: 83.6902075	total: 33.2

308:	learn: 71.4980307	total: 59.3s	remaining: 3m 48s
309:	learn: 71.4452852	total: 59.5s	remaining: 3m 48s
310:	learn: 71.3714532	total: 59.7s	remaining: 3m 48s
311:	learn: 71.3173017	total: 59.9s	remaining: 3m 48s
312:	learn: 71.2307170	total: 1m	remaining: 3m 47s
313:	learn: 71.1035314	total: 1m	remaining: 3m 47s
314:	learn: 71.0216595	total: 1m	remaining: 3m 47s
315:	learn: 70.9402261	total: 1m	remaining: 3m 47s
316:	learn: 70.8785795	total: 1m	remaining: 3m 46s
317:	learn: 70.8062723	total: 1m	remaining: 3m 46s
318:	learn: 70.7342508	total: 1m 1s	remaining: 3m 46s
319:	learn: 70.6508529	total: 1m 1s	remaining: 3m 46s
320:	learn: 70.5856824	total: 1m 1s	remaining: 3m 46s
321:	learn: 70.5091533	total: 1m 1s	remaining: 3m 45s
322:	learn: 70.4429288	total: 1m 1s	remaining: 3m 45s
323:	learn: 70.4126031	total: 1m 2s	remaining: 3m 45s
324:	learn: 70.3625218	total: 1m 2s	remaining: 3m 45s
325:	learn: 70.3123658	total: 1m 2s	remaining: 3m 45s
326:	learn: 70.1923930	total: 1m 2s	remaining:

459:	learn: 62.9833258	total: 1m 29s	remaining: 3m 21s
460:	learn: 62.9301753	total: 1m 29s	remaining: 3m 21s
461:	learn: 62.8855956	total: 1m 29s	remaining: 3m 21s
462:	learn: 62.8357094	total: 1m 29s	remaining: 3m 21s
463:	learn: 62.7629981	total: 1m 30s	remaining: 3m 21s
464:	learn: 62.7274837	total: 1m 30s	remaining: 3m 21s
465:	learn: 62.6781290	total: 1m 30s	remaining: 3m 20s
466:	learn: 62.6410505	total: 1m 30s	remaining: 3m 20s
467:	learn: 62.5717005	total: 1m 30s	remaining: 3m 20s
468:	learn: 62.5223324	total: 1m 31s	remaining: 3m 20s
469:	learn: 62.5010598	total: 1m 31s	remaining: 3m 20s
470:	learn: 62.4648159	total: 1m 31s	remaining: 3m 20s
471:	learn: 62.4189690	total: 1m 31s	remaining: 3m 19s
472:	learn: 62.3458545	total: 1m 31s	remaining: 3m 19s
473:	learn: 62.2868517	total: 1m 32s	remaining: 3m 19s
474:	learn: 62.2374169	total: 1m 32s	remaining: 3m 19s
475:	learn: 62.1850041	total: 1m 32s	remaining: 3m 19s
476:	learn: 62.1527898	total: 1m 32s	remaining: 3m 19s
477:	learn

611:	learn: 56.4078182	total: 2m	remaining: 2m 54s
612:	learn: 56.3246747	total: 2m	remaining: 2m 54s
613:	learn: 56.2944306	total: 2m	remaining: 2m 54s
614:	learn: 56.2332685	total: 2m	remaining: 2m 53s
615:	learn: 56.2072319	total: 2m 1s	remaining: 2m 53s
616:	learn: 56.1861193	total: 2m 1s	remaining: 2m 53s
617:	learn: 56.1725854	total: 2m 1s	remaining: 2m 53s
618:	learn: 56.1111420	total: 2m 1s	remaining: 2m 52s
619:	learn: 56.0459845	total: 2m 1s	remaining: 2m 52s
620:	learn: 55.9725999	total: 2m 1s	remaining: 2m 52s
621:	learn: 55.9391369	total: 2m 1s	remaining: 2m 52s
622:	learn: 55.9253217	total: 2m 2s	remaining: 2m 51s
623:	learn: 55.8965864	total: 2m 2s	remaining: 2m 51s
624:	learn: 55.8757331	total: 2m 2s	remaining: 2m 51s
625:	learn: 55.8046311	total: 2m 2s	remaining: 2m 51s
626:	learn: 55.7603219	total: 2m 2s	remaining: 2m 51s
627:	learn: 55.7132658	total: 2m 2s	remaining: 2m 50s
628:	learn: 55.6866009	total: 2m 3s	remaining: 2m 50s
629:	learn: 55.6578264	total: 2m 3s	rema

762:	learn: 50.9793289	total: 2m 29s	remaining: 2m 23s
763:	learn: 50.9325960	total: 2m 29s	remaining: 2m 23s
764:	learn: 50.9010727	total: 2m 29s	remaining: 2m 23s
765:	learn: 50.8777169	total: 2m 29s	remaining: 2m 23s
766:	learn: 50.8625549	total: 2m 29s	remaining: 2m 23s
767:	learn: 50.8323593	total: 2m 29s	remaining: 2m 22s
768:	learn: 50.8154197	total: 2m 29s	remaining: 2m 22s
769:	learn: 50.7867234	total: 2m 30s	remaining: 2m 22s
770:	learn: 50.7522435	total: 2m 30s	remaining: 2m 22s
771:	learn: 50.7012840	total: 2m 30s	remaining: 2m 21s
772:	learn: 50.6756322	total: 2m 30s	remaining: 2m 21s
773:	learn: 50.6426678	total: 2m 30s	remaining: 2m 21s
774:	learn: 50.6208443	total: 2m 30s	remaining: 2m 21s
775:	learn: 50.5835665	total: 2m 31s	remaining: 2m 21s
776:	learn: 50.5417205	total: 2m 31s	remaining: 2m 20s
777:	learn: 50.5162554	total: 2m 31s	remaining: 2m 20s
778:	learn: 50.4802714	total: 2m 31s	remaining: 2m 20s
779:	learn: 50.4459322	total: 2m 31s	remaining: 2m 20s
780:	learn

914:	learn: 46.6114023	total: 2m 57s	remaining: 1m 53s
915:	learn: 46.5816316	total: 2m 57s	remaining: 1m 53s
916:	learn: 46.5686193	total: 2m 58s	remaining: 1m 53s
917:	learn: 46.5535450	total: 2m 58s	remaining: 1m 53s
918:	learn: 46.5335186	total: 2m 58s	remaining: 1m 52s
919:	learn: 46.5148611	total: 2m 58s	remaining: 1m 52s
920:	learn: 46.4873173	total: 2m 58s	remaining: 1m 52s
921:	learn: 46.4632469	total: 2m 59s	remaining: 1m 52s
922:	learn: 46.4271411	total: 2m 59s	remaining: 1m 52s
923:	learn: 46.3988802	total: 2m 59s	remaining: 1m 51s
924:	learn: 46.3767029	total: 2m 59s	remaining: 1m 51s
925:	learn: 46.3461625	total: 2m 59s	remaining: 1m 51s
926:	learn: 46.3184916	total: 3m	remaining: 1m 51s
927:	learn: 46.2890302	total: 3m	remaining: 1m 51s
928:	learn: 46.2442131	total: 3m	remaining: 1m 50s
929:	learn: 46.2245298	total: 3m	remaining: 1m 50s
930:	learn: 46.2027310	total: 3m	remaining: 1m 50s
931:	learn: 46.1467622	total: 3m 1s	remaining: 1m 50s
932:	learn: 46.1206507	total: 3

1064:	learn: 42.8703304	total: 3m 26s	remaining: 1m 24s
1065:	learn: 42.8561344	total: 3m 26s	remaining: 1m 24s
1066:	learn: 42.8433471	total: 3m 26s	remaining: 1m 23s
1067:	learn: 42.8301601	total: 3m 26s	remaining: 1m 23s
1068:	learn: 42.8012680	total: 3m 26s	remaining: 1m 23s
1069:	learn: 42.7884879	total: 3m 27s	remaining: 1m 23s
1070:	learn: 42.7596558	total: 3m 27s	remaining: 1m 23s
1071:	learn: 42.7125632	total: 3m 27s	remaining: 1m 22s
1072:	learn: 42.6691014	total: 3m 27s	remaining: 1m 22s
1073:	learn: 42.6461339	total: 3m 27s	remaining: 1m 22s
1074:	learn: 42.6258857	total: 3m 28s	remaining: 1m 22s
1075:	learn: 42.5967922	total: 3m 28s	remaining: 1m 22s
1076:	learn: 42.5785091	total: 3m 28s	remaining: 1m 21s
1077:	learn: 42.5666143	total: 3m 28s	remaining: 1m 21s
1078:	learn: 42.5512339	total: 3m 28s	remaining: 1m 21s
1079:	learn: 42.5359019	total: 3m 28s	remaining: 1m 21s
1080:	learn: 42.4880919	total: 3m 29s	remaining: 1m 21s
1081:	learn: 42.4614484	total: 3m 29s	remaining:

1214:	learn: 39.7605793	total: 3m 53s	remaining: 54.8s
1215:	learn: 39.7386083	total: 3m 53s	remaining: 54.6s
1216:	learn: 39.7240773	total: 3m 54s	remaining: 54.4s
1217:	learn: 39.7057481	total: 3m 54s	remaining: 54.2s
1218:	learn: 39.6906867	total: 3m 54s	remaining: 54s
1219:	learn: 39.6687426	total: 3m 54s	remaining: 53.9s
1220:	learn: 39.6289305	total: 3m 54s	remaining: 53.7s
1221:	learn: 39.6124192	total: 3m 55s	remaining: 53.5s
1222:	learn: 39.6036102	total: 3m 55s	remaining: 53.3s
1223:	learn: 39.5857708	total: 3m 55s	remaining: 53.1s
1224:	learn: 39.5708830	total: 3m 55s	remaining: 52.9s
1225:	learn: 39.5656857	total: 3m 55s	remaining: 52.7s
1226:	learn: 39.5549065	total: 3m 55s	remaining: 52.5s
1227:	learn: 39.5448804	total: 3m 56s	remaining: 52.3s
1228:	learn: 39.5362318	total: 3m 56s	remaining: 52.1s
1229:	learn: 39.5187666	total: 3m 56s	remaining: 51.9s
1230:	learn: 39.5025168	total: 3m 56s	remaining: 51.7s
1231:	learn: 39.4918569	total: 3m 56s	remaining: 51.5s
1232:	learn:

1366:	learn: 36.9266185	total: 4m 20s	remaining: 25.3s
1367:	learn: 36.9125219	total: 4m 20s	remaining: 25.1s
1368:	learn: 36.8963260	total: 4m 20s	remaining: 24.9s
1369:	learn: 36.8881827	total: 4m 20s	remaining: 24.7s
1370:	learn: 36.8726345	total: 4m 20s	remaining: 24.6s
1371:	learn: 36.8355018	total: 4m 21s	remaining: 24.4s
1372:	learn: 36.8117392	total: 4m 21s	remaining: 24.2s
1373:	learn: 36.7825003	total: 4m 21s	remaining: 24s
1374:	learn: 36.7688383	total: 4m 21s	remaining: 23.8s
1375:	learn: 36.7489159	total: 4m 21s	remaining: 23.6s
1376:	learn: 36.7226717	total: 4m 21s	remaining: 23.4s
1377:	learn: 36.7019434	total: 4m 22s	remaining: 23.2s
1378:	learn: 36.6774206	total: 4m 22s	remaining: 23s
1379:	learn: 36.6596901	total: 4m 22s	remaining: 22.8s
1380:	learn: 36.6411945	total: 4m 22s	remaining: 22.6s
1381:	learn: 36.6336737	total: 4m 22s	remaining: 22.4s
1382:	learn: 36.6141030	total: 4m 22s	remaining: 22.2s
1383:	learn: 36.5988658	total: 4m 23s	remaining: 22.1s
1384:	learn: 3

In [108]:
y_preds=model.predict(X_test)
rms = mean_squared_error(y_test, y_preds, squared=False)
rms

98.61015495656022

#### Submission 

In [109]:
test_df = test_df[main_cols]
predictions = model.predict(test_df)

In [110]:
sub_file = Submission.copy()
sub_file.target = predictions
# Create a csv file and upload to zindi 
sub_file.to_csv('Baseline.csv', index = False)
sub_file.head()

,ID,target
0,test_id_0,12.672525
1,test_id_1,16.174329
2,test_id_2,16.159477
3,test_id_3,-30.418357
4,test_id_4,8.190520
